<a href="https://colab.research.google.com/github/pradiptars/ML_PricePrediction_Final/blob/main/HousePrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [1]:
from google.colab import files
uploaded = files.upload()

Saving oh_data_clean.csv to oh_data_clean.csv


In [4]:
df = pd.read_csv('oh_data_clean.csv')
df.head(2)

,bath,balcony,price,total_sqft_int,bhk,Price_per_sqft,area_type_Carpet Area,area_type_Plot Area,area_type_Super built-up Area,availability_17-Aug,...,location_Yelahanka,location_Yelahanka New Town,location_Yeshwanthpur,location_akshaya nagar t c palya,location_bsk 6th stage 2ad block near sri conversation hall,location_kamanahalli main road,location_kg halli jalhalli west,location_singapura paradise,location_t.c palya,location_vinayakanagar
0,6.0,1.584376,370.0,1020.0,6,36274.509804,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,1.000000,23.1,660.0,2,3500.000000,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df.drop('price', axis=1)
y = df['price']

In [6]:
print(X.shape)
print(y.shape)

(1067, 514)
(1067,)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
lr = LinearRegression()
lr_lasso = Lasso()
lr_ridge = Ridge()


In [11]:
def rsme(y_test, y_pred):
  return np.sqrt(mean_squared_error(y_test, y_pred))

In [12]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
lr_score = lr.score(X_test, y_test)
lr_rmse = rsme(y_test, y_pred)
lr_score, lr_rmse

(0.2682142373380624, 79.80947151996595)

In [13]:
lr_lasso.fit(X_train, y_train)
y_pred = lr_lasso.predict(X_test)
lr_lasso_score = lr_lasso.score(X_test, y_test)
lr_lasso_rmse = rsme(y_test, y_pred)
lr_lasso_score, lr_lasso_rmse

(0.3374245668873277, 75.94166622490492)

In [14]:
lr_ridge.fit(X_train, y_train)
y_pred = lr_ridge.predict(X_test)
lr_ridge_score = lr_ridge.score(X_test, y_test)
lr_ridge_rmse = rsme(y_test, y_pred)
lr_ridge_score, lr_ridge_rmse

(0.31377569846340747, 77.28505210391413)

In [15]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)
svr_score = svr.score(X_test, y_test)
svr_rmse = rsme(y_test, y_pred)
svr_score, svr_rmse

(-0.04305758033539919, 95.28325810545783)

In [16]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
rf_score = rf.score(X_test, y_test)
rf_rmse = rsme(y_test, y_pred)
rf_score, rf_rmse

(0.955875055235667, 19.597674696857073)

In [17]:
import xgboost
xgb = xgboost.XGBRegressor()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
xgb_score = xgb.score(X_test, y_test)
xgb_rmse = rsme(y_test, y_pred)
xgb_score, xgb_rmse

(0.9821864408827498, 12.451962578245343)

# Dictionary

# compair

In [19]:
print(pd.DataFrame([{'Model': 'Linierregration', 'Score': lr_score, 'RSME' : lr_rmse},
                    {'Model': 'Lasso', 'Score': lr_lasso_score, 'RSME' : lr_lasso_rmse},
                    {'Model': 'Ridge', 'Score': lr_ridge_score, 'RSME' : lr_ridge_rmse},
                    {'Model': 'SupportVactor', 'Score': svr_score, 'RSME' : svr_rmse},
                    {'Model': 'RandomForest', 'Score': rf_score, 'RSME' : rf_rmse},
                    {'Model': 'XGboost', 'Score': xgb_score, 'RSME' : xgb_rmse}],
                   columns=['Model', 'Score', 'RSME']))

             Model     Score       RSME
0  Linierregration  0.268214  79.809472
1            Lasso  0.337425  75.941666
2            Ridge  0.313776  77.285052
3    SupportVactor -0.043058  95.283258
4     RandomForest  0.955875  19.597675
5          XGboost  0.982186  12.451963


# Cross Validation

In [21]:
from sklearn.model_selection import cross_val_score, KFold
cvs = cross_val_score(xgb, X_train, y_train, cv=10)
cvs, cvs.mean()

(array([0.99619603, 0.99156807, 0.88846595, 0.9870597 , 0.96045856,
        0.9945407 , 0.93715959, 0.88460955, 0.97692487, 0.9935886 ]),
 0.9610571617315357)

In [23]:
cvs_rsme = cross_val_score(svr, X_train, y_train, cv=10)
cvs_rsme, cvs_rsme.mean()

(array([-0.02836206,  0.02518286, -0.04913512, -0.00025545, -0.17787146,
        -0.00765987, -0.13666978, -0.00768383, -0.01030721, -0.00899733]),
 -0.040175924255986685)

In [24]:
import joblib
joblib.dump(xgb, 'house_price.pkl')

['house_price.pkl']